&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../../START_HERE.ipynb)




# Challenge - Bike rental prediction

## 1. Introduction

This notebook walks through an end-to-end GPU machine learning workflow where cuDF is used for processing the data and cuML is used to train machine learning models on it. 

After completing this excercise, you will be able to use cuDF to load data from disk, combine tables, scale features, use one-hote encoding and even write your own GPU kernels to efficiently transform feature columns. Additionaly you will learn how to pass this data to cuML, and how to train ML models on it. The trained model is saved and it will be used for prediction.

It is not required that the user is familiar with cuDF or cuML. Since our aim is to go from ETL to ML training, a detailed introduction is out of scope for this notebook. We recommend [Introduction to cuDF](../../CuDF/01-Intro_to_cuDF.ipynb) for additional information.

### 1.2. Problem statement

We are trying to predict daily demand for short-term bike rentals made in 2011 and 2012. We will combine three data sources: bike rental information, historical weather data, and dates of public holidays. In Section 2 of this notebook we will use cuDF to combine these data into a single dataset that can be used as an input for machine learning algorithms.  In Section 3 we train models using cuML to predict bike rentals.

### 1.3 Why RAPIDS?

Using the GPU accelerated libraries from RAPIDS greatly reduces the execution time of a data science workflow. This leads to faster iteration with data preparation and model selection, and overall a more efficient workflow. 

### 1.2.1 References

This notebook is inspired by the [blog article](https://medium.com/rapids-ai/essential-machine-learning-with-linear-models-in-rapids-part-1-of-a-series-992fab0240da) from Paul Mahler and its accompanying [notebook](https://github.com/rapidsai-community/notebooks-contrib/blob/master/blog_notebooks/regression/regression_blog_notebook.ipynb). The dataset is prepared along the steps given by *Hadi Fanaee-T and Joao Gama* in their [paper](https://doi.org/10.1007/s13748-013-0040-3) *Event labeling combining ensemble detectors and background knowledge*. The exploratory data analysis notebooks by [Vivek Srinivasan](https://www.kaggle.com/viveksrinivasan/eda-ensemble-model-top-10-percentile) and [Mitesh Yadav](https://www.kaggle.com/miteshyadav/comprehensive-eda-with-xgboost-top-10-percentile) provided useful input for this excercise. 

First part of this notebook contains sections from [Introduction to cuDF](https://github.com/rapidsai-community/notebooks-contrib/blob/master/getting_started_notebooks/intro_tutorials/02_Introduction_to_cuDF.ipynb) by Paul Hendricks, which gives a concise introduction to cuDF, also discussing a few points  not mentioned in this notebook. 

Dataset sources:
- The bike sharing dataset is provided by [Capital Bike Share](https://www.capitalbikeshare.com/system-data).
- The weather data is retrieved from the [Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset) hosted by the UCI Machine Learning repository.
- The original source of the weather data is https://www.freemeteo.com.



<!--Two more introductory notebooks with excercises is presented by Zahra Ronaghi 
[Into to cuDF](https://github.com/zronaghi/nasa-ml-workshop/blob/master/cudf/01-Intro_to_cuDF.ipynb) and
[Intro to cuML](https://github.com/zronaghi/nasa-ml-workshop/blob/master/cuml/01-LinearRegression-Hyperparam.ipynb).
-->





## 2. Prepare dataset with cuDF
Let's start by loading the necessary libraries

In [1]:
import cudf
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import importlib
import utils
importlib.reload(utils)
from utils import fetch_bike_dataset, fetch_weather_dataset, read_bike_data_pandas
import matplotlib.pyplot as plt
import seaborn as sns

### 2.1 Prepare weather data
First, we will download the weather data.

In [ ]:
filename = fetch_weather_dataset()

cuDF DataFrames are a tabular structure of data that reside on the GPU. We interface with these cuDF DataFrames in the same way we interface with Pandas DataFrames that reside on the CPU - with a few deviations. Load data from CSV file into a cuDF DataFrame.

In [ ]:
weather = cudf.read_csv(filename)

#### 2.1.1 Inspecting a cuDF DataFrame

There are several ways to inspect a cuDF DataFrame. The first method is to enter the cuDF DataFrame directly into the REPL. This shows us an overview about the DatFrame including its type and metadata such as the number of rows or columns.

In [ ]:
weather

A second way to inspect a cuDF DataFrame is to wrap the object in a Python print function `print(weather)` function. This results in showing the rows and columns of the dataframe with simple formating.

For very large dataframes, we often want to see the first couple rows. We can use the `head` method of a cuDF DataFrame to view the first N rows.

In [ ]:
weather.head(3)

#### 2.1.2 Columns

cuDF DataFrames store metadata such as information about columns or data types. We can access the columns of a cuDF DataFrame using the `.columns` attribute.

In [ ]:
print(weather.columns)

We can modify the columns of a cuDF DataFrame by modifying the `columns` attribute. We can do this by setting that attribute equal to a list of strings representing the new columns. Let's shorten the two longest column names!

In [ ]:
### TODO rename the relative temperature column to RTemp, and the relative humidity to Humidity
weather.columns = ['Hour', 'Temperature', 'Relative Temperature', 'Rel. Humidity', 'Wind', 'Weather']
weather.head()

#### 2.1.3 Series

cuDF DataFrames are composed of rows and columns. Each column is represented using an object of type `Series`. For example, if we subset a cuDF DataFrame using just one column we will be returned an object of type `cudf.dataframe.series.Series`.

In [ ]:
humidity = weather['Humidity']
print(type(humidity))
print(humidity)

We also see a column of values on the left hand side with values 0, 1, 2, 3. These values represent the index of the Series.
The DataFrame and Series objects have both an index attribute that will be useful for joining tables and also for selecting data.

#### 2.1.4 Data Types

We can also inspect the data types of the columns of a cuDF DataFrame using the `dtypes` attribute.

In [ ]:
print(weather.dtypes)

We can modify the data types of the columns of a cuDF DataFrame by passing in a cuDF Series with a modified data type.

In [ ]:
weather['Humidity'] = weather['Humidity'].astype(np.float64)
print(weather.dtypes)

The 'Weather' column provides a description of the weather condidions. We should mark it as a categorical column.

In [ ]:
weather['Weather'] = weather['Weather'].astype('category')
weather['Weather']

After this step the numerical category codes can be accessed using the `.cat.codes` attribute of the column. We actually will not need the category labels, we just replace the 'Weather' column with the category codes.

In [ ]:
weather['Weather'] = weather['Weather'].cat.codes

The data type of the 'Hour' column is `object` which means a string. Let's convert this to a numeric value! This cannot be done with the `astype` method, you should use the [cudf.to_datetime](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.to_datetime) function!

In [ ]:
### TODO convert the 'Hour' column from string to datetime
weather['Hour'] = 
weather.dtypes

#### 2.1.2 Prepare features
##### Operations with cudf Series
We can perform mathematical operations on the Series data type. We will scale the Humidity and and Temperature variables, so that they lay in the [0, 1] range (some ML algorithms work better if the input data is scaled this way).

In [ ]:
weather['Humidity'] = weather['Humidity'] / 100.0

We will scale the temperature using the following formula T = (T - Tmin) / (Tmax - Tmin). First we select the min and max values.

In [ ]:
T = weather['Temperature']

# Select the minimum temperature
Tmin = T.min()

### TODO select the maximum temperature (1 line of code)
Tmax = 

print(Tmin, Tmax)

We could simply use the Tmin and Tmax values and apply the above formula on the series. 

##### User defined functions (UDF)
We can write custom functions to operate on the data. When cuDF executes a UDF, it gets just-in-time (JIT) compiled into a CUDA kernel (either explicitly or implicitly) and is run on the GPU. Let's write a function that scales the temperature!

In [ ]:
def scale_temp(T):
    # Note that the Tmin and Tmax variables are stored during compilation time and remain constant afterwards
    T = (T - Tmin) / (Tmax - Tmin)
    return T 

The applymap function will call scale_temp on all element of the series

In [ ]:
weather['Temperature'] = weather['Temperature'].applymap(scale_temp)

Lets do the same min-max scaling for the wind data

In [ ]:
### TODO calculate the minimum and maximum values of the 'Wind' column (2 lines of code)


### TODO define a scale_wind function and apply it on the Wind column (~ 2-3 lines of code)
def scale_wind

# apply the scale_wind function on the 'Wind' column
weather['Wind'] = weather['Wind'].applymap(scale_wind)

Let's inspect the table, the Temperature, Wind and Humidity columns should have values in the [0, 1] range.

In [ ]:
weather.describe()

##### Dropping Columns

The relative temperature column is correlated with the temperature, it will not give much extra information for the ML model. We want to remove this column from our `DataFrame`. We can do so using the `drop_column` method. Note that this method removes a column in-place - meaning that the `DataFrame` we act on will be modified.

In [ ]:
weather.drop_column('RTemp')
weather

If we want to remove a column without modifying the original DataFrame, we can use the `drop` method. This method will return a new DataFrame without that column (or columns).

##### Index

Like `Series` objects, each `DataFrame` has an index attribute.

In [ ]:
weather.index

We can use the index values to subset the `DataFrame`. Lets use this to plot the first 48 values. Before plotting we have to transfer from the GPU memory to the system memory. We use the `to_array` method to return a copy of the data as a numpy array.

In [ ]:
selection = weather[weather.index<48]
plt.plot(selection['Hour'].to_array(), selection['Temperature'].to_array())
plt.xlabel('Hour')
plt.ylabel('Temperature [C]')

We can also change the index. Our dataset has one entry for each hour, so one could set the 'Hour' coulmn as index by calling
```
weather = weather.set_index('Hour')
```

We do not perform this change right now, but we will use it later.

### 2.2 Prepare bike sharing data
We start by downloading the data

In [ ]:
files = fetch_bike_dataset([2011,2012])

Let's read the first file to have an idea of the dataset

In [ ]:
cudf.read_csv(files[0])

We are only interested in the events when a bicicle was rented. Let us read the first column from all files, by specifying the `usecols` argument to [read_csv](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.io.csv.read_csv). We can use the `parse_dates` argument to parse the date string into a datetime variable, or the [to_datetime](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.to_datetime) function that we have used for the weather dataset. After all the tables are read we will concatenate them.

Note: one has to specify a list of columns [ column1, column2 ] for the `usecol` argument.

In [ ]:
def read_bike_data(files):
    # Reads a list of files and concatenates them
    tables = []
    for filename in files:
        ### TODO read column 1 ('Start date') from the CSV file, and convert it to datetime format
        ### (1-2 lines of code)
        tmp_df = 
        
        ### END TODO
        tables.append(tmp_df) 
        
    merged_df = cudf.concat(tables, ignore_index=True)
    
    # Sanity checks
    if merged_df.columns != ['Start date']:
        raise ValueError("Error incorrect set of columns read")
    if merged_df['Start date'].dtype != 'datetime64[ns]':
        raise TypeError("Stard date should be converted to datetime type")
        
    return merged_df

We will also measure the execution time of reading and processing the data (you can execute the cell multiple times to have a better measurement).

In [ ]:
%%time
bikes_raw = read_bike_data(files)

For comparision, we can repeat the same operation on the CPU. We have prepared a helper function for that.

In [ ]:
%%time
bikes_raw_pd = read_bike_data_pandas(files)

We want to count the number of rental events in every hour. We will define a new feature where we remove the minutes and seconds part of the time stamp. Since pandas has a convenient `floor` function defined to do it, we will convert the column to a pandas Series, transform it with the floor operation, and then put it back on the GPU.

In [ ]:
bikes_raw['Hour'] = bikes_raw['Start date'].to_pandas().dt.floor('h')

We will aggregate the number of bicicle rental events for each hour. We use the [groupby](https://docs.rapids.ai/api/cudf/nightly/api.html#groupby) function.

In [ ]:
bikes = bikes_raw.groupby('Hour').agg('count')
bikes.columns = ['cnt']
bikes.head(5)

Let's add a column to the new dataset: the date without the time of the day. We can derive that similarly to the 'Hour' feature above. After the groupby operation, the 'Hour' became the index of the dataset, we will apply the `floor` operation on the index. 

In [ ]:
bikes['date'] = bikes.index.to_pandas().floor('D')

It will be usefull to define a set of additional features: hour of the day, day of month, month and year https://docs.rapids.ai/api/cudf/nightly/api.html#datetimeindex

In [ ]:
bikes['hr'] = bikes.index.hour

### TODO add year and month features (~ 2 lines of code)
bikes['year'] = 
bikes['month'] = 

Remove the offset from year

In [ ]:
bikes['year'] = bikes['year'] - 2011

#### Visualize data
It is a good practice to visulize the data. We will have to use the to_array() method to convert the cudF Series objects to numpy arrays that can be plotted.

In [ ]:
### TODO plot bikes['cnt'] 
plt.plot(bikes.index.to_array(), )

It is hard to see much apart from the global trend. Let's have a look how the 'cnt' variable looks like as a function the 'month' and 'hr' features. We will use [boxplot](https://seaborn.pydata.org/generated/seaborn.boxplot.html) from the Seaborn package.

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2)
fig.set_size_inches(12, 5)
sns.boxplot(data=bikes.to_pandas(), y="cnt",x="month",orient="v",ax=axes[0])
sns.boxplot(data=bikes.to_pandas(), y="cnt",x="hr",orient="v",ax=axes[1])
axes[0].set(xlabel='Months', ylabel='Count',title="Box Plot On Count Across months")
axes[1].set(xlabel='Hour Of The Day', ylabel='Count',title="Box Plot On Count Across Hour Of The Day")
plt.show()

#### 3.2.1 Combine weather data with bike rental data

In [ ]:
gdf_bw = bikes.merge(weather, left_index=True, right_on='Hour', how='inner')

# inspect the merged table
gdf_bw

We can see that the data is not sorted after the merge use the [sort_values](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.core.dataframe.DataFrame.sort_values) method to

In [ ]:
### TODO sort the table according to the index (1 line of code)
gdf_bw = gdf_bw.sort_values(by='Hour')

# Inspect the sorted table
gdf_bw

### 3.3 Add working day feature

Apart from the weather, in important factor that influences people's daily activities is whether it is a working day or not. In this section we will create a working day feature. First we add the weekday as a new feature column. 
We can use the [weekday](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.core.series.DatetimeProperties.weekday) attribute of the [datetime](https://docs.rapids.ai/api/cudf/nightly/api.html#datetimeindex)

In [ ]:
gdf_bw['Weekday'] = gdf_bw['date'].dt.weekday

Next create a table with all the holidays in Washington DC in 2011-2011

In [ ]:
holidays = cudf.DataFrame({'date': ['2011-01-17', '2011-02-21', '2011-04-15', '2011-05-30', '2011-07-04', '2011-09-05', '2011-11-11', '2011-11-24', '2011-12-26', '2012-01-02', '2012-01-16', '2012-02-20', '2012-04-16', '2012-05-28', '2012-07-04', '2012-09-03', '2012-11-12', '2012-11-22', '2012-12-25'],
'Description': ["Martin Luther King Jr. Day", "Washington's Birthday", "Emancipation Day", "Memorial Day", "Independence Day", "Labor Day", "Veterans Day", "Thanksgiving", "Christmas Day", 
"New Year's Day", "Martin Luther King Jr. Day", "Washington's Birthday", "Emancipation Day", "Memorial Day", "Independence Day", "Labor Day", "Veterans Day", "Thanksgiving", "Christmas Day"]})

# Print the dataframe
holidays

We convert the date from string to datetime type, and drop the description column. Additionally we add a new column marked 'Holiday'. This will be useful to mark the holidays after we merge the tables.

In [ ]:
holidays['date'] = cudf.to_datetime(holidays['date'])
holidays.drop_column('Description')
holidays['Holiday'] = 1
holidays

Now we are ready to merge the tables using the commond `date` column. We want keep every element from the gdf_bw table (our *left* table), so we use a left join. Hint: use [merge](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.core.dataframe.DataFrame.merge) with the `on` and `how` attributes

In [ ]:
### TODO merge tables and on the column 'date', use a left merge 
gdf = gdf_bw.merge(...)

# inspect the result
gdf

We reset the index to 'Hour' and sort the table accordingly. Notice that most of the rows in the 'Holiday' column are filled with `<NA>`, only the dates that appeared in the holiday table are filled with 1. We shall fill the empty fields with zero.

In [ ]:
gdf = gdf.set_index('Hour')
gdf = gdf.sort_index()
### TODO fill empty holiday values with zero (1 line of code)
gdf['Holiday'] = 

# inspect the result
gdf

Next, we create a workingday feature. Assuming that the first five days of the week are working days, one could do that simply with the following operation:
```
gdf['Workingday'] = (gdf['Weekday'] < 5) & (gdf['Holiday']!=1)
```
But we could do it with user defined functions too. Previously we have only used UDF to process elements of a series. Now we will process rows of a dataframe and
combine the 'Weekday' and 'Holiday' columns to calculate the new feature 'Workingday'.

More on user defined functions in our [blog](https://medium.com/rapids-ai/user-defined-functions-in-rapids-cudf-2d7c3fc2728d) and in the [documentation](https://docs.rapids.ai/api/cudf/nightly/guide-to-udfs.html).

In [ ]:
def workday_kernel(Weekday, Holiday, Workingday):
    for i, (w, h) in enumerate(zip(Weekday, Holiday)):
        # variable w will take values from the Weekday column
        # variable h will take values from the Holiday column
        ### TODO add the working day condition using variables w and h (1 line of code)
        Workingday[i] =

In [ ]:
gdf = gdf.apply_rows(workday_kernel, incols=['Weekday', 'Holiday'], outcols=dict(Workingday=np.float64), kwargs=dict())

After this step we will not need the 'Holiday' and 'date' columns, we can drop them

In [ ]:
gdf = gdf.drop(['Holiday', 'date'])

### 2.4 One-hot encoding

We have all now the data in a single table, but we still want to change their encoding. We're going to create one-hot encoded variables, also known as dummy variables, for each of the time variables as well as the weather situation.


A summary from https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/:

"The integer values have a natural ordered relationship between each other and machine learning algorithms may be able to understand and harness this relationship.
For categorical variables where no such ordinal relationship exists, the integer encoding is not enough.

In fact, using this encoding and allowing the model to assume a natural ordering between categories may result in poor performance or unexpected results (predictions halfway between categories).

In this case, a one-hot encoding can be applied to the integer representation. This is where the integer encoded variable is removed and a new binary variable is added for each unique integer value.
"

We start by one-hot encoding the 'Weather' column using the [one_hot_encoding](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.core.dataframe.DataFrame.one_hot_encoding) method from cuDF DataFrame. This is very similar to the [get_dummies](https://docs.rapids.ai/api/cudf/nightly/api.html#cudf.core.reshape.get_dummies) function (which might be more familiar for Pandas users), but one_hot_encoding works on a single input column.

In [ ]:
codes = gdf['Weather'].unique()
gdf = gdf.one_hot_encoding('Weather', 'Weather_dummy', codes)
# Inspect the results
gdf.head(3)

We're going to drop the original variable as well as one of the new dummy variables so we don't create colinearity (more about this problem [here](https://towardsdatascience.com/one-hot-encoding-multicollinearity-and-the-dummy-variable-trap-b5840be3c41a)).

In [ ]:
gdf = gdf.drop(['Weather', 'Weather_dummy_1'])

We create a copy of the dataset. It will make it easier to start over in case something would go wrong during the next excercise. 

In [ ]:
gdf_backup = gdf.copy()

In [ ]:
dummies_list = ['month', 'hr', 'Weekday']

gdf = gdf_backup.copy()

for item in dummies_list:
    ### TODO implement one-hot encoding for item (~ 2 lines of code)
    
    
    ### TODO drop one of the nowly added dummy variable (~ 1 line of code)
    gdf.drop_column(...)
    
    gdf.drop_column(item) # drop the original item
    
# Inspect the resulting table
gdf

### 2.5 Save the prepared dataset

In [ ]:
gdf.to_csv('data/bike_sharing.csv')

## 3. Predict bike rentals with cuML

cuML is a GPU accelerated machine learning library. cuML's Python API mirrors the [Scikit-Learn](https://scikit-learn.org/stable/) API.

cuML currently requires all data be of the same type, so this loop converts all values into floats

In [ ]:
import cuml

In [ ]:
for col in gdf.columns:
    gdf[col] = gdf[col].astype('float64')

### 3.1 Prepare training and test data
It is customary to denote the input feature matrix with X, and the target that we want to predict with y. We separete the target column 'cnt' from the rest of the table.

In [ ]:
y = gdf['cnt']
X = gdf.drop('cnt')

Let's split the data randomly into a train and a test set

In [ ]:
X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(X, y)

### 3.2 Linear regression
Our first model is a simple linear regression, which tries to predict the output as a linear combination of the input features:

<img src="./img/regression_eq.png" width="150px">



In [ ]:
reg = cuml.LinearRegression()
reg.fit(X_train, y_train)

We can make prediction with the trained data

In [ ]:
y_hat = reg.predict(X_test)

We can visualize the how well the model works. Let's plot data for may 2012:

In [ ]:
import datetime as dt
def plot_timerange(model, start, end):
    start_date = dt.datetime.strptime(start, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end, '%Y-%m-%d')
    idx = (X.index >= start_date) & (X.index <= end_date)
    X_may = X[idx]
    y_may = y[idx]

    ### TODO predict the values for X_may (1 line of code)
    y_pred = 

    plt.plot(X_may.index.to_array(), y_may.to_array())
    plt.plot(X_may.index.to_array(), y_pred.to_array(), '--')

In [ ]:
plot_timerange(reg, '2011-06-01', '2011-06-05')

Often we are interested in a single score. The default score method for regression problems is the [r2_score](https://en.wikipedia.org/wiki/Coefficient_of_determination).

In [ ]:
train_score = reg.score(X_train, y_train)
### TODO calculate test score (the score on X_test, ~ 1 line of code)
test_score = 

print('train score', train_score)
print('test score', test_score)

### 3.3  Save and load the trained model
We can pickle any cuML model

In [ ]:
import pickle
pickle_file = 'my_model.pickle'

with open(pickle_file, 'wb') as pf:
    pickle.dump(reg, pf)

Load the saved model

In [ ]:
with open(pickle_file, 'rb') as pf:
        loaded_model = pickle.load(pf)

print('Loaded model   score', loaded_model.score(X_test, y_test))
print('Original model score', reg.score(X_test, y_test))

### 3.4 Ridge regression with hyperparameter tuning
Ridge regression is a linear regression model with an added L2 regularization term. Regularization is often used in practice to avoid overfitting. The strength of the regularization is set by the alpha hyperparameter. 
We're going to do a small hyperparameter search for alpha, checking 100 different values. This is fast to do with RAPIDS. Also notice that we are appending the results of each Ridge model onto the dictionary containing our earlier results, so we can more easily see which model is the best at the end. 

In [ ]:
output = {'score_OLS': test_score}

for alpha in np.arange(0.01, 1, 0.01): #alpha value has to be positive    
    ridge = cuml.Ridge(alpha=alpha, fit_intercept=True)
    ### TODO fit the model and calculate the test score (2 lines of code)
    
    score = 
    ### END EXCERCISE ###
    output['score_RIDGE_{}'.format(alpha)] = score

Here we see that our regulaized model does better than the rest, include OLS with all the variables. 

In [ ]:
print('Max score: {}'.format(max(output, key=output.get)))

### 3.5 Additional cuML models (Optional)
#### 3.5.1 Support vector regression

Support vector regression is a more complex model, with an execution time that scales with at least O(n_rows^2). RAPIDS cuML includes a fast SVM solver that makes it feasable to run SVM on larger datasets.

In [ ]:
%%time
reg = cuml.svm.SVR(kernel='rbf', gamma=0.1, C=100, epsilon=0.1)
reg.fit(X_train, y_train)
reg.score(X_train, y_train)
reg.score(X_test, y_test)

We can also use sklearns [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to perform hyperparameter search. Sklearn's GridSearchCV requires input that the input data is a host array. Fortunately cuML is flexible with the [input format](https://medium.com/rapids-ai/input-and-output-configurability-in-rapids-cuml-e719d72c135b), and we can pass numpy array directly to it (at a cost of additional host to device copies, because under the hood cuML copies the data to the GPU). If the data size is reasonably small, then we can pay the price of additional data movement and  combine the convenience of GridSearchCV with the speed of cuML algorithms.

In [ ]:
param_grid = [ {'C': [0.01, 0.1, 1, 10, 100], 'gamma': [10, 1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} ]

In [ ]:
X_train_np = X_train.as_matrix()
y_train_np = y_train.to_array()
X_test_np = X_test.as_matrix() 
y_test_np = y_test.to_array()

In [ ]:
from sklearn.model_selection import GridSearchCV
reg = GridSearchCV(cuml.svm.SVR(), param_grid, scoring='r2' )

reg.fit(X_train_np, y_train_np)

print("Best parameters set found on development set:")
print()
print(reg.best_params_)
print()
print("Grid scores on development set:")
print()
means = reg.cv_results_['mean_test_score']
stds = reg.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

#### 3.5.2 KNN Regression
k-Nearest Neighbors regression is a machine learning technique that predicts an unknown observation by using the k most similar known observations in the training dataset.

In [ ]:
%%time
### TODO tune the n_neighbors hyperparameter to achieve better performance
knn = cuml.neighbors.KNeighborsRegressor(n_neighbors=8)
knn.fit(X_train, y_train, convert_dtype=True)
pred = knn.predict(X_test)
knn.score(X_test, y_test)

We can compare the execution time of training KNN with cuML and with scikit-learn

In [ ]:
import sklearn
import sklearn.neighbors

In [ ]:
%%time
knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=8)
knn.fit(X_train_np, y_train_np,)
pred = knn.predict(X_test_np)
knn.score(X_test_np, y_test_np)

### 3.6 XGBoost (Optional)
RAPIDS integrates seamlessly with the XGBoost library. Here is how to use it for our example

In [ ]:
import xgboost as xgb

In [ ]:
xgr=xgb.XGBRegressor(max_depth=8,min_child_weight=6,gamma=0.4)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# instantiate params
params = {}

# general params
general_params = {}
params.update(general_params)

# booster params
booster_params = {'tree_method': 'gpu_hist'}
params.update(booster_params)

# learning task params
learning_task_params = {'objective': 'reg:squarederror'}
params.update(learning_task_params)
print(params)

In [ ]:
evallist = [(dtest, 'test'), (dtrain, 'train')]
num_round = 100

In [ ]:
bst = xgb.train(params, dtrain, num_round, evallist)


In [ ]:
y_pred = bst.predict(dtest)

In [ ]:
import cupy as cp
from cuml.metrics.regression import r2_score
y_pred_cp = cp.asarray(y_pred)
y_test_cp = cp.asarray(y_test).astype(np.float32)

In [ ]:
r2_score(y_test_cp, y_pred_cp)

In [ ]:
plot_timerange(reg, '2011-06-01', '2011-06-05')

#### Conclusion

Write down your observations and compare the scores. Complete the exercise and code while obtaining your optimal solution, and refer to the hints provided in it. We hope that you found this exercise exciting and beneficial in understanding RAPIDS better. Share your highest accuracy and try to use the unique features of RAPIDS for accelerating your data science pipelines. Don't restrict yourself to the previously explained concepts, but use the documentation to apply more models and functions and achieve the best results.

## Licensing
  
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;
[Home Page](../../START_HERE.ipynb)
